In [1]:
from unityagents import UnityEnvironment
import numpy as np
from agent import Agent
from collections import deque
import matplotlib.pyplot as plt
import torch

In [2]:
env = UnityEnvironment(file_name="./Tennis_Windows_x86_64/Tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

# examine dones
print('dones:', env_info.local_done)

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]
dones: [False, False]


In [5]:
agents = [Agent(state_size=state_size, action_size=action_size, num_agents=num_agents, agent_id = i, random_seed=1) for i in range(num_agents)]

In [6]:
agents[0].actor_local.load_state_dict(torch.load('checkpoint_actor_0.pth'))
agents[1].actor_local.load_state_dict(torch.load('checkpoint_actor_1.pth'))

In [7]:
for i in range(10):

    env_info = env.reset(train_mode=False,)[brain_name] # reset enviroment and turn off training mode
    states = env_info.vector_observations               # get first states from the resetted enviroment
    
    for agent in agents:
        agent.reset() 
    
    scores = np.zeros(num_agents) 

    t = 0
    
    while True:


        actions = np.concatenate(np.array([agent.act(states, add_noise=False) for agent in agents]), axis=0)        # select an action

        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished

        states = next_states                               # roll over states to next time step
        scores += np.array(rewards)                          # update the score (for each agent)

        print('\rep:{:02}\tscore:{: .2f}'.format(i, np.max(scores)), end="")
        
        t += 1

        if np.any(dones):                                  # exit loop if episode finished                
            break

    print('\rep:{:02}\tscore:{: .2f}'.format(i, np.max(scores)))


ep:00	score: 0.70
ep:01	score: 2.30
ep:02	score: 0.40
ep:03	score: 0.40
ep:04	score: 0.70
ep:05	score: 0.40
ep:06	score: 0.40
ep:07	score: 0.80
ep:08	score: 0.40
ep:09	score: 0.50


In [8]:
env.close()